# CALCULATING SYSTEM LCOE 
This example demonstrates one method of calculating the LCOE of a system, if the energy yield output of the system is known.

In this case, we assume artificial reflectors were incorporated in the single-axis-tracked bifacial PV system, and incorporate those costs in the LCOE calculation. We compare the LCOE when the cost of the material is included with the LCOE if these costs are ignored.

### Initialize a scene:
Set initial variables and import libraries. Set input file where energy yield is stored.


In [1]:
import pandas as pd
import numpy as np
import sys
import os

""" Set up system parameters """
module_power = 350 #power rating of module in watts
module_width = 1 #assume 1 meter module width
module_xgap = 0.05 #assume 5 cm spacing along torque tube

""" Set up LCOE parameters """
# set up LCOE and Module power variables here
i = 0.015 #1.5% interest rate
n = 30 #30 year project lifetime

""" Set up Reflector parameters """
reflector_length = 2.85 #length in meters in N-S direction (runs length of rows E-W), in m
#calculate the reflector area per watt of PV module power
reflector_area_perW = reflector_length*(module_width+module_xgap)/module_power

""" Read energy yield calculation results """
results_file = './VF_power.csv'   
if not os.path.isfile(results_file):
    sys.exit(str(results_file)+': results file does not exist')

### Import power from results file and calculate total energy yield.
Assume hourly data. A full year of energy yield is standard when calculating the capacity factor (CF).
Assume that the power in the results file is for a single module (rated at the input module power).

In [2]:
# Use pandas here to read energy yield results file
data = pd.read_csv(results_file)

# Assuming hourly data, find number of hours in input file
num_hours = len(data.index)
# print('number of hours: '+str(num_hours) + ' hr')

#Take hourly energy yield (assume hourly data) and sum to get total energy yield 
total_EY = data['Pout'].sum()
print('total energy yield: '+str(total_EY) + ' Wh')


total energy yield: 658412.710357724 Wh


### Perform LCOE Calculations
#### Assume system and maintenance costs
First, input PV system costs and maintenance costs. Here, utility-scale single-axis-tracking costs from [1] are used. We add a "bifaciality premium" as suggested in [2].  

In [3]:
""" LCOE Calculations """
#LCOE method taken from NREL simplified in [3]

#PV system costs - CAPEX and O&M
PV_System_price_perW = 0.89 + 0.015 # cost in $/Wdc for system from [1], plus additional bifaciality cost from [2]
O_M_annual_perW = 16.06/1000 #O&M costs in $/wdc/yr from [1]


#### Calculate the capacity factor
The capacity factor (CF) is a number between 0 and 1 that represents the total energy output over time, divided by the system capacity times the total number of hours. 

It is expressed as:

$CF = \frac{\text{total energy yield}}{\text{(rated system capacity)(total hours)}}$

For an annual calculation, this can be:

$CF = \frac{\text{annual energy yield}}{\text{(rated system capacity)(8760)}}$

In [4]:
#Capacity factor for reference system
CF = (total_EY/(module_power*num_hours))
print('CF: '+str(CF))

CF: 0.21474648087336073


#### Calculate the capital recovery factor
The capital recovery factor (CRF) is a method to take up-front costs (such as PV system price) and convert it to an equivalent annual cost, assuming a loan over n years with an interest rate i. In other words, the CRF is the ratio of an annuity over n years to the present value of that annuity. The CRF is expressed as:

$CRF = \frac{i(1+i)^n}{(1+i)^n-1}$

In [5]:
#Calculate capital recovery factor [3]
CRF = (i*np.power(1+i,n))/(np.power(1+i,n)-1)
print('CRF: '+str(CRF))

CRF: 0.04163918826153069


#### Calculate the levelized cost of energy (LCOE)
Lastly, we calculate the levelized cost of energy (LCOE). The LCOE is the cost, typically expressed in cents per kWh, of generating electricity for a given system.

$LCOE = \frac{\text{sum of costs over lifetime}}{\text{sum of energy produced over lifetime}}$

Here, we use the simple LCOE method described in [3]. 

$LCOE = \frac{(\text{total capital costs }\times\text{ CRF}) + (\text{annual O&M costs})}{\text{(8760 hours)}\times CF}$

In [6]:
#Calculate simplified levelized cost of energy (LCOE) [3]
LCOE = 1000*100*((PV_System_price_perW*CRF) + O_M_annual_perW)/(8760*CF)
print('LCOE without reflector cost: '+str(LCOE) +' cents/kWh')

LCOE without reflector cost: 2.856903061853107 cents/kWh


#### Calculate LCOE with Reflector included
Repeat the LCOE calculation with the cost of the reflector included in the calculation.


In [8]:
#Reflector costs - assume no increase to O&M costs
reflector_cost_perm2 = 0.1 #Assume 0.10$/m2#cost of reflecting material per m2
reflector_cost_perW = reflector_area_perW*reflector_cost_perm2

#Total CAPEX costs
total_cost_perW = PV_System_price_perW + reflector_cost_perW

#Calculate simplified levelized cost of energy (LCOE) [3]
LCOE_reflectors = 1000*100*((total_cost_perW*CRF) + O_M_annual_perW)/(8760*CF)
print('LCOE with reflector cost: '+str(LCOE_reflectors) +' cents/kWh')

LCOE with reflector cost: 2.8587955719597984 cents/kWh


### References
[1] V. Ramasamy, D. Feldman, J. Desai, and R. Margolis, “U.S. Solar Photovoltaic System and Energy Storage Cost Benchmark: Q1 2021,” 2021. Available: https://www.nrel.gov/docs/fy22osti/80694.pdf.

[2] D. Feldman et al., “Spring 2022 Solar Industry Update,” 2022. Available: https://www.nrel.gov/docs/fy22osti/82854.pdf.

[3] National Renewable Energy Laboratory, “Simple Levelized Cost of Energy (LCOE) Calculator Documentation.” https://www.nrel.gov/analysis/tech-lcoe-documentation.html